<a href="https://colab.research.google.com/github/nvhien1961999/Covid19_auto_detecting_vision_transformer_model/blob/main/train_VGG16_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive._mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
root_folder = '/content/gdrive/MyDrive/VGG' + '/'

In [ ]:
# khoi tao toc do hoc ban dau so lan lap va so anh moi lan lap
INIT_LR = 1e-3
EPOCHS=50

In [ ]:
# import thu vien
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import cv2
import os
from os import listdir
import pickle
import time
from tensorflow.keras.callbacks import ModelCheckpoint

In [ ]:
def load_data(src):
    src = src + '/'
    #resize
    print("[INFO] loading images...")

    data = []
    labels = []
    num_img = 0

    # Lặp qua các folder con trong thư mục raw
    for folder in listdir(src):
        if folder!='.DS_Store':
            print("Folder: ",folder)
            # Lặp qua các file trong từng thư mục chứa các anh
            for file in listdir(src  + folder):
                if file!='.DS_Store':
                    print("File: ", file + ", folder: ",folder)
                    data.append( cv2.resize(cv2.imread(src + folder + "/" + file),dsize=(224,224)))
                    labels.append( folder)
                    num_img += 1

    data = np.array(data)
    labels = np.array(labels)#.reshape(-1,1)

    from sklearn.preprocessing import LabelBinarizer
    encoder = LabelBinarizer()
    labels = encoder.fit_transform(labels)
    print(labels)

    return data, labels

In [ ]:
trainX, trainY = load_data('/content/gdrive/MyDrive/dataset/splitted_dataset/train')
valX, valY = load_data('/content/gdrive/MyDrive/dataset/splitted_dataset/val')

In [ ]:
# tao doi tuong de tang du lieu huan luyen
trainAug = ImageDataGenerator(
	rotation_range=15,
	fill_mode="nearest"
  )

In [ ]:
def get_model():
  # load mang VGG16 vo hieu cac lop FC
  baseModel = VGG16(weights="imagenet", include_top=False, input_tensor=Input(shape=(224, 224, 3)))
  headModel = baseModel.output
  headModel = AveragePooling2D(pool_size=(4, 4))(headModel)
  headModel = Flatten(name="flatten")(headModel)
  headModel = Dense(64, activation="relu")(headModel)
  headModel = Dropout(0.5)(headModel)
  headModel = Dense(3, activation="softmax")(headModel)
  # place the head FC model on top of the base model (this will become
  # the actual model we will train)
  model = Model(inputs=baseModel.input, outputs=headModel)
  for layer in baseModel.layers:
    layer.trainable = False
  print("[INFO] compiling model...")
  opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
  model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy"])
  print("[INFO] training head...")
  return model

In [ ]:
model = get_model()
filepath="weights-{epoch:02d}-{val_accuracy:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

58900480/58889256 [==============================] - 2s 0us/step
[INFO] compiling model...
[INFO] training head...


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [ ]:
start = time.time()
vgghist = model.fit_generator(
    trainAug.flow(trainX,trainY),
    validation_data=(valX,valY),
    epochs=50,
    callbacks=callbacks_list
)
end = time.time()
print("Training time: ", end - start)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  


Epoch 1/50
324/324 [==============================] - ETA: 0s - loss: 0.6463 - accuracy: 0.5385

In [ ]:
model.save(root_folder + "filepath.hdf5")

In [ ]:
import json
# Get the dictionary containing each metric and the loss for each epoch
history_dict = vgghist.history
# Save it under the form of a json file
json.dump(history_dict, open(root_folder + "vgghist.history", 'w'))

In [ ]:
print(trainX.shape)
print(trainY.shape)
print(valX.shape)
print(valY.shape)

In [ ]:
f = open(root_folder + "training_time.txt", "w")
f.write(str(end-start))
f.close()